In [10]:
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
import numpy as np

from keras import Sequential
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import Adam

In [17]:
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same", input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same")) 
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))
    # compile model
    adam_opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=adam_opt, metrics=['accuracy'])
    return model

def load_and_norm_samples():
    (train_data, _), (_, _) = load_data()
    training_images = np.expand_dims(train_data, axis = -1)
    training_images = training_images.astype('float32')
    training_images = training_images / 255.0
    return training_images
    
def generate_real_images(dataset, num_samples):
    idx = np.random.randint(0, dataset.shape[0], num_samples)
    gen_data = dataset[idx]
    gen_labels = np.ones((num_samples, 1))
    return gen_data, gen_labels
    
def generate_fake_images(num_samples):
    gen_data = np.random.rand(28 * 28 * num_samples)
    gen_data = gen_data.reshape((num_samples, 28, 28, 1))
    gen_labels = np.zeros((num_samples, 1))
    return gen_data, gen_labels

def train_discriminator(model, dataset, num_periods=100, batch_size=256):
    half_batch_size = int(batch_size / 2)
    for epoch in range(num_periods):
        image_real, label_real = generate_real_images(dataset, half_batch_size)
        _, real_acc = model.train_on_batch(image_real, label_real)
        image_fake, label_fake = generate_fake_images(half_batch_size)
        _, fake_acc = model.train_on_batch(image_fake, label_fake)
        print('>%d real=%.0f%% fake=%.0f%%' % (epoch, real_acc, fake_acc))

In [18]:
model = define_discriminator()
dataset = load_and_norm_samples()

train_discriminator(model, dataset)

>0 real=1% fake=1%
>1 real=1% fake=1%
>2 real=1% fake=1%
>3 real=1% fake=1%
>4 real=1% fake=1%
>5 real=1% fake=1%
>6 real=1% fake=1%
>7 real=1% fake=1%
>8 real=1% fake=1%
>9 real=1% fake=1%
>10 real=1% fake=1%
>11 real=1% fake=1%
>12 real=1% fake=1%
>13 real=1% fake=1%
>14 real=1% fake=1%
>15 real=1% fake=1%
>16 real=1% fake=1%
>17 real=1% fake=1%
>18 real=1% fake=1%
>19 real=1% fake=1%
>20 real=1% fake=1%
>21 real=1% fake=1%
>22 real=1% fake=1%
>23 real=1% fake=1%
>24 real=1% fake=1%
>25 real=1% fake=1%
>26 real=1% fake=1%
>27 real=1% fake=1%
>28 real=1% fake=1%
>29 real=1% fake=1%
>30 real=1% fake=1%
>31 real=1% fake=1%
>32 real=1% fake=1%
>33 real=1% fake=1%
>34 real=1% fake=1%
>35 real=1% fake=1%
>36 real=1% fake=1%
>37 real=1% fake=1%
>38 real=1% fake=1%
>39 real=1% fake=1%
>40 real=1% fake=1%
>41 real=1% fake=1%
>42 real=1% fake=1%
>43 real=1% fake=1%
>44 real=1% fake=1%
>45 real=1% fake=1%
>46 real=1% fake=1%
>47 real=1% fake=1%
>48 real=1% fake=1%
>49 real=1% fake=1%
>50 real=1